###### Aula 06 - Perceptron

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.metrics import accuracy_score

### Iris DataSet

In [2]:
iris = datasets.load_iris()
print("dir(iris) =", dir(iris))
print("iris.feature_names =", iris.feature_names)
print("iris.target_names =", iris.target_names)
print("iris.target =", iris.target)
print("iris.data =", iris.data)

dir(iris) = ['DESCR', 'data', 'feature_names', 'target', 'target_names']
iris.feature_names = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
iris.target_names = ['setosa' 'versicolor' 'virginica']
iris.target = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
iris.data = [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2

In [3]:
def randomize(a):
    np.random.seed(7483)
    np.random.shuffle(a)

In [4]:
randomize(iris.data)
randomize(iris.target)
X = iris.data
y = iris.target

### Classe do Perceptron

O perceptron é uma unidade básica de aprendizado por máquina, que possui várias entradas ligadas diretamente a uma saída.

In [5]:
class Perceptron:
    
    def __init__(self, learn_rate=0.1, max_iter=100, max_error=0.001):
        self.learn_rate = learn_rate
        self.max_iter = max_iter
        self.max_error = max_error
        self.w = None
        return
    
    def fit(self, X, y):
        features = len(X[0]) + 1
        self.w = np.random.rand(features)*2 - 1
        
        for i in range(0, self.max_iter):
            
            # calculando o erro associado aos parâmetros
            res = self._pred(X, self.w)
            error = sum(abs(y - res))
            #print(self.w, y, res, error)
            if (error <= self.max_error):
                return

            # calculando o vetor gradiente, que indicará
            # em que direção o erro mais diminui
            grad = np.zeros(features)
            wc = np.array(self.w, copy=True)
            for f in range(0, features):
                dw = 1e-8
                prev = wc[f]
                wc[f] += dw
                res2 = self._pred(X, wc)
                error2 = sum(abs(y - res2))
                de = error2 - error
                d = de/dw
                grad[f] = d
                wc[f] = prev
            norm = np.linalg.norm(grad)
            vec = grad/norm

            # na verdade quero que a taxa de variação seja inversa
            # quanto a norma do gradiente, ou seja:
            # - se o erro varia muito, quero dar passos pequenos
            # - se o erro varia pouco, quero dar passos grandes
            step_vec = -vec/norm*self.learn_rate

            self.w += step_vec
        
        return
    
    def predict(self, X):
        return (self._pred(X, self.w) >= 0.5)*1
    
    def _pred(self, X, w):
        results = np.zeros(len(X))
        for i in range(0, len(X)):
            results[i] = self._activation(np.dot(X[i], w[0:-1]) + w[-1])
        return results
    
    def _activation(self, x):
        return np.exp(-np.logaddexp(0, -x))

### Particionamento
Irei particionar os dados em N partes, sendo que:
- 1/N será de teste
- (N-1)/N será de aprendizado

In [6]:
p = Perceptron(learn_rate=0.5)

X2 = [
    [1.0,2.0,3.0],
    [1.1,2.1,3.1],
    [-1.0,-2.0,-3.0],
    [-1.1,-2.1,-3.1]]
y2 = np.array([1,1,0,0])
p.fit(X2,y2)

X_new = [
    [0.8,1.5,2.0],
    [-0.8,-1.5,-2.0]
]
p.predict(X_new)

array([1, 0])

In [7]:
N = 5
for i in range(0, N):
    X_treino = []
    y_treino = []
    X_teste = X[i::N]
    y_teste = y[i::N]
    for j in range(0, N):
        if (i != j):
            X_treino.extend(X[j::N])
            y_treino.extend(y[j::N])
    
    p0 = Perceptron()
    y0_treino = (np.array(y_treino) == 0)*1
    y0_teste = (np.array(y_teste) == 0)*1
    print(y0_treino, y0_teste)
    
    p0.fit(X_treino, y0_treino)
    y0_pred = p0.predict(X_teste)
    acc = accuracy_score(y0_teste, y0_pred)
    print("teste:",y0_teste,"pred:", y0_pred)


[1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0
 1 1 0 0 0 1 1 0 0] [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0]


c:\python\python36-32\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide
c:\python\python36-32\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in logaddexp
c:\python\python36-32\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in greater_equal


teste: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0] pred: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0
 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0
 1 1 0 0 0 1 1 0 0] [1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0]
teste: [1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0] pred: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0
 1 1 0 0 0 1 1 0 0] [0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0]
teste: [0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0] pred: [0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 0 